# Creation des datasets
### Note : remplacer les "filter_VX"

In [1]:
import os, sys

p = os.path.abspath('..')
sys.path.insert(1, p)

In [8]:
import os,glob
import random
import cv2
import shutil
from tqdm import tqdm
from modules import filters

In [4]:
# A changer
origin = "3DPrintingAugmented"
filter_version = "V1"

In [5]:
# Ne pas modifier
destination = origin+"_"+filter_version
remove_inverse = False

# Remplacer les jpg par des jpeg (Impératif)

In [6]:
from PIL import Image
import os
print(origin)
compteur=0
for filename in os.listdir(origin):
    if filename.endswith(".jpg"):
        im = Image.open(origin+'/'+filename)
        name=origin+'/'+filename[:-4]+'.jpeg'
        im.save(name)
        os.remove(origin + '/' + filename)
        compteur+=1
        continue
    else:
        continue
print(compteur," images ont été changées")

3DPrintingAugmented
0  images ont été changées


# Visualisation du filtre (optionnal) avant la création du dataset

In [7]:
dirpath = os.path.join(os.getcwd(), origin+'/*.jpeg')
images_path = glob.glob(dirpath)
import re

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

images_path.sort(key=natural_keys)
for img_path in images_path:
    print(img_path)
    # Loading image and filtering versions
    
    img = cv2.imread(img_path)
    filters.display_labels(img,img_path)
    cv2.imshow("original",img)
    
    filtered_image = filters.filter_V1(img_path)
    filters.display_labels(filtered_image,img_path)
    cv2.imshow("V1",filtered_image)

    
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cv2.waitKey(1)

/Users/dorianvoydie/Documents/YOLO/Dataset/3DPrintingAugmented/image_0.jpeg


-1

# Création des dossiers

In [14]:
os.system("mkdir "+destination)
os.system("mkdir "+destination+"/Valset")
os.system("mkdir "+destination+"/Trainset")
os.system("mkdir "+destination+"/Testset")
os.system("touch "+destination+"/config.yaml")
os.system("echo 'train: ../{}/Trainset' > ".format(destination)+destination+"/config.yaml")
os.system("echo 'val: ../{}/Valset' >> ".format(destination)+destination+"/config.yaml")
os.system("echo 'nc: 1' >> ".format(destination)+destination+"/config.yaml")
os.system("echo 'names: [\"Scratch\"]' >> ".format(destination)+destination+"/config.yaml")

0

# Remove inverse (optionnal)

In [15]:
# if remove_inverse==True:
#   compteur = 0
#   for filename in glob.glob(os.path.join(origin+'/*')):
#     if "_I" in filename:
#       compteur+=1
#       os.remove(filename) 
#   print(compteur," inverses ont été enlevés")

# Path

In [16]:
# Images path
destination_images = os.path.join(origin+'/*.jpeg')
destination_labels = os.path.join(origin+'/*.txt')

In [17]:
images_path = glob.glob(destination_images)
random.shuffle(images_path)
print(len(images_path))
labels_path = glob.glob(destination_labels)
print(len(labels_path))

2720
2720


In [18]:
from sklearn.model_selection import train_test_split
trainset_images, testset_images = train_test_split(images_path, test_size=0.1, shuffle=True)
trainset_images, valset_images = train_test_split(trainset_images, test_size=0.2, shuffle=True)
print("Il y a ",len(trainset_images)," images d'entrainement")
print("Il y a ",len(valset_images)," images de validation")
print("Il y a ",len(testset_images)," images de test")

Il y a  1958  images d'entrainement
Il y a  490  images de validation
Il y a  272  images de test


# Répartition

In [19]:
sets = {"Valset":valset_images,"Trainset":trainset_images,"Testset":testset_images}
for image_set_name,image_set_value in sets.items():
  for img_path in tqdm(image_set_value):
    image_name = img_path.split("/")[1]
    # Loading image
    filtered = eval("filters.filter_{}(img_path)".format(filter_version))
    # Getting the label associated
    label_associated = image_name[:-5]+'.txt'
    img_new_path = destination+'/'+str(image_set_name)+'/'+image_name
    cv2.imwrite(img_new_path, filtered)
    label_original = origin+'/'+label_associated
    label_target = destination+'/'+str(image_set_name)+'/'+label_associated
    shutil.copyfile(label_original, label_target)

100%|██████████| 272/272 [00:01<00:00, 203.03it/s]
